In [1]:
# !pip install mlxtend
# !pip install pandas 
# !pip install pprintpp

In [31]:
#!pip install --upgrade gradio==4.21.0 tomlkit==0.12.0
#!pip install --upgrade streamlit==1.16.0 protobuf<4,>=3.12
#!pip install --upgrade gensim FuzzyTM>=0.4.0


In [1]:
pip show numpy

Name: numpy
Version: 1.24.4
Summary: Fundamental package for array computing in Python
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD-3-Clause
Location: c:\users\sunit\onedrive\desktop\mygithub_pjts\coffeeshop_chatbot\venv\lib\site-packages
Requires: 
Required-by: contourpy, matplotlib, mlxtend, pandas, scikit-learn, scipy
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import association_rules, apriori

# Read Dataset

In [3]:
sales_reciepts = pd.read_csv ('dataset/201904 sales reciepts.csv')
sales_reciepts.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [4]:
product = pd.read_csv ('dataset/product.csv')
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


# Data Wrangling

### Merge Data

In [5]:
dataset = pd.merge(sales_reciepts[['transaction_id','transaction_date', 'sales_outlet_id', 'customer_id', 'product_id', 'quantity']],
                      product[['product_id', 'product_category', 'product']],
                      on ='product_id', how = 'left' )
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


### Remove Sizes

In [6]:
dataset[dataset['product'].str.contains('Dark chocolate')]['product'].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [7]:
dataset['product'].nunique()

80

In [8]:
dataset['product']= dataset['product'].str.replace(' Rg','')
dataset['product']= dataset['product'].str.replace(' Sm','')
dataset['product']= dataset['product'].str.replace(' Lg','')

In [9]:
dataset['product'].nunique()

45

In [10]:
dataset['product'].unique()

array(['Traditional Blend Chai', 'Brazilian', 'Serenity Green Tea',
       'Our Old Time Diner Blend', 'Jamaican Coffee River', 'Ethiopia',
       'English Breakfast', 'Sustainably Grown Organic', 'Earl Grey',
       'Cappuccino', 'Espresso shot', 'Latte', 'Dark chocolate',
       'Columbian Medium Roast', 'Oatmeal Scone', 'Morning Sunrise Chai',
       'Peppermint', 'Jumbo Savory Scone', 'Lemon Grass',
       'Chocolate Chip Biscotti', 'Spicy Eye Opener Chai',
       'Ginger Biscotti', 'Chocolate Croissant', 'Hazelnut Biscotti',
       'Cranberry Scone', 'Scottish Cream Scone ', 'Croissant',
       'Almond Croissant', 'Ginger Scone', 'Ouro Brasileiro shot',
       'Organic Decaf Blend', 'Chocolate syrup', 'Hazelnut syrup',
       'Carmel syrup', 'Sugar Free Vanilla syrup',
       'Jamacian Coffee River', 'Guatemalan Sustainably Grown',
       'Civet Cat', 'Chili Mayan', 'Primo Espresso Roast',
       'Brazilian - Organic', 'I Need My Bean! Diner mug',
       'Espresso Roast', 'I Need 

### Choose Product Subset

In [11]:
products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]

In [12]:
dataset = dataset[dataset['product'].isin(products_to_take)]

In [13]:
dataset['product'].nunique()

18

In [14]:
dataset[['product','product_category']].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Latte,Coffee
3,Dark chocolate,Drinking Chocolate
4,Oatmeal Scone,Bakery
5,Jumbo Savory Scone,Bakery
6,Chocolate Chip Biscotti,Bakery
7,Ginger Biscotti,Bakery
8,Chocolate Croissant,Bakery
9,Hazelnut Biscotti,Bakery


### Clean Transactions

In [15]:
#Concatenate transaction id with customer id and store in a new transaction column
dataset['transaction'] = dataset['transaction_id'].astype(str) +"_"+  dataset['customer_id'].astype(str)

In [16]:
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino,127_116
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131


In [17]:
num_of_items_for_each_transaction = dataset['transaction'].value_counts().reset_index()
num_of_items_for_each_transaction.head()

,transaction,count
0,209_0,31
1,206_0,30
2,204_0,27
3,208_0,25
4,203_0,24


In [19]:
#Get list of transactions with single count ie only buying 1 item--> not required for recommendation engine
valid_transactions = num_of_items_for_each_transaction[(num_of_items_for_each_transaction['count']>1)]['transaction'].tolist()
valid_transactions

['209_0',
 '206_0',
 '204_0',
 '208_0',
 '203_0',
 '207_0',
 '205_0',
 '151_0',
 '323_0',
 '301_0',
 '42_0',
 '9_0',
 '76_0',
 '202_0',
 '213_0',
 '307_0',
 '274_0',
 '149_0',
 '46_0',
 '51_0',
 '370_0',
 '242_0',
 '298_0',
 '195_0',
 '197_0',
 '475_0',
 '6_0',
 '329_0',
 '344_0',
 '317_0',
 '102_0',
 '59_0',
 '54_0',
 '325_0',
 '172_0',
 '255_0',
 '310_0',
 '118_0',
 '319_0',
 '223_0',
 '140_0',
 '132_0',
 '3_0',
 '97_0',
 '36_0',
 '216_0',
 '282_0',
 '314_0',
 '10_0',
 '135_0',
 '320_0',
 '146_0',
 '278_0',
 '313_0',
 '75_0',
 '211_0',
 '43_0',
 '234_0',
 '122_0',
 '144_0',
 '291_0',
 '171_0',
 '266_0',
 '253_0',
 '356_0',
 '251_0',
 '61_0',
 '352_0',
 '421_0',
 '334_0',
 '228_0',
 '284_0',
 '30_0',
 '315_0',
 '187_0',
 '345_0',
 '377_0',
 '38_0',
 '289_0',
 '239_0',
 '164_0',
 '268_0',
 '143_0',
 '152_0',
 '410_0',
 '295_0',
 '327_0',
 '311_0',
 '227_0',
 '170_0',
 '177_0',
 '260_0',
 '45_0',
 '85_0',
 '32_0',
 '359_0',
 '280_0',
 '235_0',
 '190_0',
 '306_0',
 '262_0',
 '264_0',
 '4

In [20]:
dataset = dataset[dataset['transaction'].isin(valid_transactions)]
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
34,199,2019-04-01,3,112,41,2,Coffee,Cappuccino,199_112
35,199,2019-04-01,3,112,79,1,Bakery,Jumbo Savory Scone,199_112
54,296,2019-04-01,3,328,39,1,Coffee,Latte,296_328
55,296,2019-04-01,3,328,79,1,Bakery,Jumbo Savory Scone,296_328
64,357,2019-04-01,3,530,41,2,Coffee,Cappuccino,357_530


In [21]:
dataset.shape

(10189, 9)

## Product Trends

In [22]:
dataset["product_category"].value_counts()

product_category
Bakery                3800
Coffee                3174
Flavours              2246
Drinking Chocolate     947
Packaged Chocolate      22
Name: count, dtype: int64

In [23]:
dataset['product'].value_counts()

product
Cappuccino                  1290
Latte                       1256
Dark chocolate               969
Chocolate Croissant          636
Espresso shot                628
Sugar Free Vanilla syrup     605
Chocolate syrup              568
Carmel syrup                 561
Hazelnut syrup               512
Ginger Scone                 417
Jumbo Savory Scone           357
Croissant                    355
Chocolate Chip Biscotti      352
Cranberry Scone              350
Almond Croissant             347
Hazelnut Biscotti            338
Oatmeal Scone                334
Ginger Biscotti              314
Name: count, dtype: int64

# Popularity Recommendation Engine

In [24]:
popularity_recommendation = dataset.groupby(['product','product_category']).count().reset_index()#All columns show the no of counts for the occurance of product
popularity_recommendation

,product,product_category,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,transaction
0,Almond Croissant,Bakery,347,347,347,347,347,347,347
1,Cappuccino,Coffee,1290,1290,1290,1290,1290,1290,1290
2,Carmel syrup,Flavours,561,561,561,561,561,561,561
3,Chocolate Chip Biscotti,Bakery,352,352,352,352,352,352,352
4,Chocolate Croissant,Bakery,636,636,636,636,636,636,636
5,Chocolate syrup,Flavours,568,568,568,568,568,568,568
6,Cranberry Scone,Bakery,350,350,350,350,350,350,350
7,Croissant,Bakery,355,355,355,355,355,355,355
8,Dark chocolate,Drinking Chocolate,947,947,947,947,947,947,947
9,Dark chocolate,Packaged Chocolate,22,22,22,22,22,22,22


In [25]:
#Taking one of the columns as all columns are populated with same value of counts
popularity_recommendation = popularity_recommendation[['product','product_category','transaction_id']]
#Since the transaction_id currently holds the no of times the product occured in transactions renaming it
popularity_recommendation = popularity_recommendation.rename(columns = {'transaction_id':'number_of_transactions'})
popularity_recommendation.head()

,product,product_category,number_of_transactions
0,Almond Croissant,Bakery,347
1,Cappuccino,Coffee,1290
2,Carmel syrup,Flavours,561
3,Chocolate Chip Biscotti,Bakery,352
4,Chocolate Croissant,Bakery,636


In [26]:
#Save dataframe as a csv
popularity_recommendation.to_csv('api/recommendation_objects/popularity_recommendation.csv',index=False)

# Apriori Recommendation Engine

In [27]:
train_basket = (dataset.groupby(['transaction','product'])['product'].count().reset_index(name ='Count'))
train_basket.head(10)

,transaction,product,Count
0,1000_0,Dark chocolate,1
1,1000_0,Oatmeal Scone,1
2,1001_8306,Cappuccino,1
3,1001_8306,Carmel syrup,1
4,1002_0,Carmel syrup,1
5,1002_0,Dark chocolate,2
6,1002_0,Espresso shot,1
7,1002_0,Ginger Scone,1
8,1004_5383,Chocolate syrup,1
9,1004_5383,Latte,1


In [28]:
my_basket = train_basket.pivot_table(index='transaction', columns='product', values='Count', aggfunc='sum').fillna(0)
my_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1005_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0


In [29]:
#Function to remove counts>1 as we require only 0s and 1s
def encode_units(x):
  if x <= 0:
    return 0
  if x >= 0:
    return 1

my_basket_sets = my_basket.applymap(encode_units)
my_basket_sets.head(10)

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1005_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1005_5559,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1006_0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1007_8375,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1008_0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [30]:
frequent_items = apriori(my_basket_sets, min_support = 0.05,use_colnames = True)#min support 5%
frequent_items.head()

c:\Users\sunit\OneDrive\Desktop\MyGithub_Pjts\CoffeeShop_Chatbot\venv\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.115646,(Almond Croissant)
1,0.388889,(Cappuccino)
2,0.191232,(Carmel syrup)
3,0.112623,(Chocolate Chip Biscotti)
4,0.135676,(Chocolate Croissant)


In [31]:
rules_basket = association_rules(frequent_items, metric = "lift", min_threshold = 1)
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Cappuccino),(Almond Croissant),0.388889,0.115646,0.053288,0.137026,1.184874,0.008314,1.024775,0.255319
1,(Almond Croissant),(Cappuccino),0.115646,0.388889,0.053288,0.460784,1.184874,0.008314,1.133333,0.176432
2,(Dark chocolate),(Almond Croissant),0.277022,0.115646,0.057445,0.207367,1.793115,0.025409,1.115717,0.611791
3,(Almond Croissant),(Dark chocolate),0.115646,0.277022,0.057445,0.496732,1.793115,0.025409,1.436567,0.500152
4,(Almond Croissant),(Latte),0.115646,0.382086,0.054422,0.470588,1.231629,0.010235,1.167171,0.212660


In [32]:
# Save Rules Basket
rules_basket.to_pickle('rules_basket.pkl')

In [33]:
# Get Example Recommendations
rules_basket[rules_basket['antecedents']=={'Latte'}].sort_values('confidence',ascending=False).head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
71,(Latte),(Sugar Free Vanilla syrup),0.382086,0.200302,0.108844,0.284866,1.422182,0.032311,1.118249,0.480415
33,(Latte),(Carmel syrup),0.382086,0.191232,0.107710,0.281899,1.474121,0.034643,1.126260,0.520509
45,(Latte),(Chocolate syrup),0.382086,0.188964,0.103175,0.270030,1.428997,0.030974,1.111053,0.485842
69,(Latte),(Hazelnut syrup),0.382086,0.171580,0.101285,0.265084,1.544961,0.035727,1.127231,0.570848
53,(Latte),(Croissant),0.382086,0.114135,0.057067,0.149357,1.308605,0.013458,1.041407,0.381651
39,(Latte),(Chocolate Croissant),0.382086,0.135676,0.055178,0.144411,1.064381,0.003338,1.010209,0.097889
67,(Latte),(Ginger Scone),0.382086,0.133409,0.055178,0.144411,1.082472,0.004204,1.012860,0.123300
5,(Latte),(Almond Croissant),0.382086,0.115646,0.054422,0.142433,1.231629,0.010235,1.031236,0.304358
64,(Latte),(Ginger Biscotti),0.382086,0.106198,0.054044,0.141444,1.331890,0.013467,1.041053,0.403272
49,(Latte),(Cranberry Scone),0.382086,0.116024,0.051398,0.134520,1.159416,0.007067,1.021371,0.222518


### Save in Json Format

In [35]:
product_categories = dataset[['product','product_category']].drop_duplicates().set_index('product').to_dict()['product_category']
product_categories

{'Cappuccino': 'Coffee',
 'Jumbo Savory Scone': 'Bakery',
 'Latte': 'Coffee',
 'Chocolate Chip Biscotti': 'Bakery',
 'Espresso shot': 'Coffee',
 'Hazelnut Biscotti': 'Bakery',
 'Chocolate Croissant': 'Bakery',
 'Dark chocolate': 'Packaged Chocolate',
 'Cranberry Scone': 'Bakery',
 'Croissant': 'Bakery',
 'Almond Croissant': 'Bakery',
 'Ginger Biscotti': 'Bakery',
 'Oatmeal Scone': 'Bakery',
 'Ginger Scone': 'Bakery',
 'Chocolate syrup': 'Flavours',
 'Hazelnut syrup': 'Flavours',
 'Carmel syrup': 'Flavours',
 'Sugar Free Vanilla syrup': 'Flavours'}

In [36]:
#Create json such that for each product we have a list of recommendations
recommendations_json = {}

antecedents = rules_basket['antecedents'].unique()
for antecedent in antecedents:
    #Extract rows for an antecedent from the pivot table
    df_rec = rules_basket[rules_basket['antecedents']==antecedent]
    #Sort values based on confidence score in decreasing order
    df_rec = df_rec.sort_values('confidence',ascending=False)
    key = "_".join(antecedent)
    recommendations_json[key] = []
    for _, row in df_rec.iterrows():
        rec_objects =row['consequents']
        for rec_object in rec_objects:
            already_exists = False
            for current_rec_object in recommendations_json[key]:
                if rec_object == current_rec_object['product']:
                    already_exists=True
            if already_exists:
                continue
            
            rec = {'product':rec_object, 
                   'product_category':product_categories[rec_object],
                   'confidence': row['confidence']
                  }
            recommendations_json[key].append(rec)

In [38]:
import pprint
pprint.pp(recommendations_json)

{'Cappuccino': [{'product': 'Sugar Free Vanilla syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.29057337220602525},
                {'product': 'Chocolate syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.2818270165208941},
                {'product': 'Carmel syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.26433430515063167},
                {'product': 'Hazelnut syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.23226433430515062},
                {'product': 'Ginger Scone',
                 'product_category': 'Bakery',
                 'confidence': 0.15354713313896987},
                {'product': 'Chocolate Chip Biscotti',
                 'product_category': 'Bakery',
                 'confidence': 0.14480077745383868},
                {'product': 'Chocolate Croissant',
                 'product_category': 'Bakery',
   

In [39]:
import json
with open('api/recommendation_objects/apriori_recommendations.json', 'w') as json_file:
    json.dump(recommendations_json, json_file)